<a href="https://colab.research.google.com/github/muniappabalaji/TimeSeries/blob/main/LLM/Agents/langgraph_basic_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langgraph langchain langchain_community langchain_experimental langchain_openai google-search-results numexpr

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 17.4 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=e744aaaf77714e1c1828e67d8dca06f15745900a96e8f77a15a984

In [2]:
import os
import operator
from typing import TypedDict, Annotated, List
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from langgraph.graph import StateGraph, END

In [18]:
os.environ["SERPAPI_API_KEY"] = "cc1f23578dbd5c3f93590b24db78a758e44f6a99eef96d1a345061554fcc79e1"

llm = ChatOpenAI(openai_api_base = "https://openrouter.ai/api/v1", openai_api_key = "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", model = "deepseek/deepseek-chat-v3.1:free",temperature=0.9) #max_tokens=100

In [19]:
# --- Tool Definition ---
# Instantiate the SerpAPI search tool.
search_tool = SerpAPIWrapper()

# --- State Definition ---
# This defines the "memory" or "state" that flows through the graph.
class ResearchState(TypedDict):
    topic: str
    explanation: str
    summary: str


In [20]:
def researcher_agent(state: ResearchState) -> dict:
    """
    This agent uses a web search tool to find information on a topic and then explains it.
    """
    print("---RESEARCHER (with SerpApi Web Search)---")
    topic = state["topic"]

    # Use the tool to search for information. The .run() method takes the query string.
    search_results = search_tool.run(topic)

    print(f"Search Results:\n{search_results}")

    prompt = ChatPromptTemplate.from_template(
        """You are a helpful research assistant. Based on the following search results,
        provide a brief, easy-to-understand explanation of the topic: {topic}.

        Search Results:
        {search_results}
        """
    )

    chain = prompt | llm
    result = chain.invoke({"topic": topic, "search_results": search_results})

    print(f"Researcher's Explanation:\n{result.content}")
    return {"explanation": result.content}

In [21]:
def summarizer_agent(state: ResearchState) -> dict:
    """
    This agent takes an explanation and summarizes it in one sentence.
    """
    print("---SUMMARIZER---")
    explanation = state["explanation"]

    prompt = ChatPromptTemplate.from_template(
        "You are a summarization expert. Condense the following text into a single, concise sentence:\n\n{explanation}"
    )

    chain = prompt | llm
    result = chain.invoke({"explanation": explanation})

    print(f"Summarizer's Output:\n{result.content}")
    return {"summary": result.content}

In [22]:
# --- Graph Definition ---

workflow = StateGraph(ResearchState)

# Add the agent functions as nodes
workflow.add_node("researcher", researcher_agent)
workflow.add_node("summarizer", summarizer_agent)

# Define the edges, which control the flow
workflow.add_edge("researcher", "summarizer")
workflow.add_edge("summarizer", END)

# Set the entry point
workflow.set_entry_point("researcher")

# Compile the graph
app = workflow.compile()

In [24]:
# --- Main Execution Block ---
if __name__ == "__main__":
    topic = input("Please enter a topic for the agents to research and summarize: ")

    inputs = {"topic": topic}
    final_state = app.invoke(inputs)

    print("\n--- FINAL SUMMARY ---")
    print(final_state['summary'])

Please enter a topic for the agents to research and summarize: Lang chain
---RESEARCHER (with SerpApi Web Search)---
Search Results:
['The platform for reliable agents. · LangChain products power top engineering teams, from startups to global enterprises · Accelerate agent development. · Ship ...', 'LangChain is a framework for building LLM-powered applications. It helps you chain together interoperable components and third-party integrations to simplify ...', 'LangChain is an open source orchestration framework for the development of applications using large language models (LLMs), like chatbots and virtual ...', 'LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration ...', 'LangChain is an open source framework for building applications based on large language models (LLMs). LLMs are large deep-learning models pre-trained on large ...']
Researcher's Explanation:
Of course! Based on the